## "Raw" notebook
This is the raw jupyter notebook used for analysis. If you wish to reproduce our results, run all cells of this notebook in a properly setup EMR notebook.

In [ ]:
# Installing newest version of pip to resolve compatibility issues
# requests for fetching terrain tile data, matplotlib for visualisation
# sc is the default spark context
sc.install_pypi_package("pip==23.2.1")
sc.install_pypi_package("requests")
sc.install_pypi_package("matplotlib")

In [ ]:
# stdlib imports
from time import time
from itertools import product
from typing import Tuple, List, NamedTuple
from collections import namedtuple

# util libraries imports
import numpy as np
import matplotlib.pyplot as plt

# PySpark related imports
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as spark_func


In [ ]:
# Constants used throughout the notebook
Tile: NamedTuple = namedtuple("Tile", ("zoom", "x", "y"))

TEMP_DIR: str = "data"
DATASET_URL_FORMAT: str = "s3://elevation-tiles-prod/terrarium"
ZOOM: int = 3

TILE_WIDTH: int = 256
TILE_HEIGHT: int = 256
CHANNELS_NUM: int = 3

RED_CHANNEL_MULTIPLIER: int = 256
GREEN_CHANNEL_MULTIPLIER: int = 1
BLUE_CHANNEL_MULTIPLIER: int = 1/256

INCREMENT_OFFSET: int = 32768

# Bounds, (latitude1, longitude1), (latitude2,  longitude2) found using bboxfinder.com
BOUNDS: Tuple[float] = (72.711037, -172.964981, -55.554805, -21.269288)


GRADIENT_GROUPS: dict[int, int] = {
    0: 2,
    1: 10,
    2: 50,
    3: 150,
    4: 500,
    5: 1500
}

## Functions related to transforming data, later used as map functions on the RDD 

In [ ]:
def mercator_projection(lat: float, lon: float, zoom: int) -> Tuple[int, int, int]:
    """
    Convert latitude, longitude to z/x/y tile coordinate at given zoom.

    Parameters:
    - lat (float): Latitude in degrees.
    - lon (float): Longitude in degrees.
    - zoom (int): Zoom level.

    Returns:
    - Tuple[int, int, int]: A tuple containing the zoom level, x and y tile coordinates.
    """
    # Convert latitude and longitude to radians
    x1: float = lon * np.pi / 180.
    y1: float = lat * np.pi / 180.

    # Mercator projection https://en.wikipedia.org/wiki/Mercator_projection
    x2 = x1
    y2 =  np.log(np.tan(0.25 * np.pi + 0.5 * y1))

    # Transform to tile space
    tiles = 2 ** zoom
    diameter =  2 * np.pi
    x3 = int(tiles * (x2 + np.pi) / diameter)
    y3 = int(tiles * (np.pi - y2) / diameter)

    return zoom, x3, y3


def get_elevation(tile: np.ndarray) -> np.ndarray:
    """
    Calculates elevation for an entire map tile.

    Parameters:
    - tile (np.ndarray): A NumPy array representing the tile.

    Returns:
    - np.ndarray: A NumPy array containing the elevation values.
    """
    return np.array([list(map(calculate_elevation, row)) for row in tile])


## Functions related to fetching data and other miscellanous transformations

In [ ]:
def get_tiles(zoom: int, lat1: float, lon1: float, lat2: float, lon2: float) -> List[Tile]:
    """
    Convert geographic bounds into a list of tile coordinates at given zoom.

    Parameters:
    - zoom (int): Zoom level.
    - lat1, lon1, lat2, lon2 (float): Geographic bounds.

    Returns:
    - List[Tile]: A list of Tile namedtuples.
    """
    # Convert to geographic bounding box
    min_lat = min(lat1, lat2)
    min_lon =  min(lon1, lon2)
    max_lat = max(lat1, lat2)
    max_lon =  max(lon1, lon2)

    # Convert to tile-space bounding box
    _, xmin, ymin = mercator_projection(max_lat, min_lon, zoom)
    _, xmax, ymax = mercator_projection(min_lat, max_lon, zoom)

    # Generate a list of tiles
    xs = range(xmin, xmax+1)
    ys = range(ymin, ymax+1)

    return [Tile(zoom, x, y) for (y, x) in product(ys, xs)]

def generate_links(tiles: List[Tile]) -> List[str]:
    """
    Returns links to s3 bucket for given tiles.

    Parameters:
    - tiles (List[Tile]): A list of Tile namedtuples.

    Returns:
    - List[str]: A list of URLs.
    """
    return [f"{DATASET_URL_FORMAT}/{tile.zoom}/{tile.x}/{tile.y}.png" for tile in tiles]


## Functions related to elevations, whether it's coloring the map or calculating color of a pixel that should be displayed

In [ ]:
def calculate_elevation(rgb: Tuple[int, int, int]) -> float:
    """
    Calculates elevation based on RGB values of a pixel.

    Parameters:
    - rgb (Tuple[int, int, int]): A tuple containing the RGB values.

    Returns:
    - float: Elevation value.
    """
    elevation = rgb[2]*RED_CHANNEL_MULTIPLIER + rgb[1]*GREEN_CHANNEL_MULTIPLIER + rgb[0]*BLUE_CHANNEL_MULTIPLIER - INCREMENT_OFFSET
    return elevation if elevation > 0 else 0

def match_to_group(elevation: float) -> int:
    """
    Matches pixel elevation to one of the predefined elevation groups.

    Parameters:
    - elevation (float): Elevation value.

    Returns:
    - int: Group ID.
    """
    for group_id, elevation_threshold in GRADIENT_GROUPS.items():
        if elevation < elevation_threshold:
            return int(group_id)

def group_elevation(tile: np.ndarray) -> np.ndarray:
    """
    Groups the elevations of an entire tile.

    Parameters:
    - tile (np.ndarray): A NumPy array representing the tile.

    Returns:
    - np.ndarray: A NumPy array containing the group IDs.
    """
    return np.array([list(map(match_to_group, np.abs(row))) for row in tile[0]])


## Functions for obtaining X and Y coordinates from images

In [ ]:
def get_x_from_image_url(image_url: str) -> int:
    """
    Extracts the tile's x-coordinate from the image path.

    Parameters:
    - image_url (str): The URL of the image.

    Returns:
    - int: The x-coordinate of the tile.
    """
    cleaned = image_url.replace(f"{DATASET_URL_FORMAT}/", "").replace(".png", "")
    return int(cleaned.split("/")[1])

def get_y_from_image_url(image_url: str) -> int:
    """
    Extracts the tile's y-coordinate from the image path.

    Parameters:
    - image_url (str): The URL of the image.

    Returns:
    - int: The y-coordinate of the tile.
    """
    cleaned = image_url.replace(f"{DATASET_URL_FORMAT}/", "").replace(".png", "")
    return int(cleaned.split("/")[2])


## De facto analysis process

In [ ]:
# Generate tile coordinates based on the specified geographic bounds and zoom level
tiles: List[Tile] = get_tiles(ZOOM, *BOUNDS)

# Generate URLs for each tile to fetch data from the S3 bucket
data_urls: List[str] = generate_links(tiles)

# Read image data into a Spark DataFrame
df = spark.read.format("image").load(data_urls)

# Select only the 'origin' and 'data' columns from the DataFrame
df = df.select("image.origin", "image.data")

# Add columns for x and y tile coordinates, derived from the 'origin' column
col_x = spark_func.udf(get_x_from_image_url, IntegerType())
col_y = spark_func.udf(get_y_from_image_url, IntegerType())
df = df.withColumn("x", col_x(df.origin))
df = df.withColumn("y", col_y(df.origin))

# Sort the DataFrame by x and y coordinates
df = df.orderBy(spark_func.col("x").asc(), spark_func.col("y").asc())

# Select only the 'data' column and convert it to an RDD
df_images = df.select("data")
tiles_rdd = df_images.rdd.map(lambda img: np.reshape(img, (TILE_HEIGHT, TILE_WIDTH, CHANNELS_NUM)))

# Calculate elevation for each tile
elevation_tiles = tiles_rdd.map(get_elevation)

# Calculate the gradient for each tile
grad_arr = elevation_tiles.map(np.gradient)

# Group elevations into predefined categories of elevations (6 categories)
grouped_arr = np.asarray(grad_arr.map(group_elevation).collect())

# Calculate the dimensions of the final world map
x_range: int = max(tiles, key=lambda tile: tile.x).x - min(tiles, key=lambda tile: tile.x).x + 1
y_range: int = max(tiles, key=lambda tile: tile.y).y - min(tiles, key=lambda tile: tile.y).y + 1

# Initialize an empty array for the world map
world_map: np.ndarray = np.zeros((y_range * TILE_HEIGHT, x_range * TILE_WIDTH))

# Populate the world map array with grouped elevation data
for x in range(x_range):
    for y in range(y_range):
        world_map[y * TILE_HEIGHT:(y + 1) * TILE_HEIGHT, x * TILE_WIDTH:(x + 1) * TILE_WIDTH] = grouped_arr[x * y_range + y, :, :]

In [ ]:
# Plot the western hemisphere map using pyplot
plt.imshow(world_map, cmap=plt.get_cmap("terrain"))
plt.xticks()
cbar = plt.colorbar()
cbar.set_ticks([0.5, 1, 2, 3, 4, 5])
cbar.set_ticklabels(["2", "10", "50", "150", "500", "1500"])

# Latitude and longituted range
lat_limits = [72.711037, -55.554805]
lon_limits = [-172.964981, -21.269288]

# Set the tick positions and labels to underline the lat/long values used
plt.xticks([0, 1023], [str(lon_limits[0]), str(lon_limits[1])])
plt.yticks([0, 1279], [str(lat_limits[0]), str(lat_limits[1])])

%matplot plt


In [ ]:
# This has to be in another cell to prevent multiple color bars from appearing when displaying the map
cbar.remove()